# CRUD PostgreSQL with python through psycopg2

### Psycopg2

    import psycopg2

### Parâmetros de conexão com o banco

    DB_HOST=''  
    POSTGRES_DB='''  
    POSTGRES_USER='''  
    POSTGRES_PASSWORD='''  

### Criando a conexão

    connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PASSWORD)

### Definindo um ponteiro

    cur = connection.cursor()

### Criando tabela

    cur.execute("CREATE TABLE [*table_name*] ([id serial PRIMARY KEY], KEY_1 VALUE_1, KEY_2 VALUE_2, KEY_3 VALUE_3,[...]);")

**OBS.:** A flag "PRIMARY KEY" pode ser usada para identifica qualquer campo como chave primária.



In [ ]:
import psycopg2
#from django.db import transaction

In [ ]:
DB_HOST='localhost'
POSTGRES_DB='pg_dev'
POSTGRES_USER='openbot'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'

connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PASSWORD)
cur = connection.cursor()

In [ ]:
cur = connection.cursor()

In [ ]:
cur.execute("CREATE TABLE person (id serial PRIMARY KEY, name text, age integer);")

In [ ]:
cur.execute("INSERT INTO person (name, age) VALUES (%s, %s)",("marcus", 35))
cur.execute("INSERT INTO person (name, age) VALUES (%s, %s)",('mariana', 26))


In [ ]:
connection.commit()

In [ ]:
cur.execute("CREATE TABLE person_teste_marcus (name text PRIMARY KEY, age integer);")

In [ ]:
cur.execute("INSERT INTO person_teste_marcus (name, age) VALUES (%s, %s)",("marcus", 35))
cur.execute("INSERT INTO person_teste_marcus (name, age) VALUES (%s, %s)",('mariana', 26))

In [ ]:
connection.commit()

In [ ]:
#Como 

In [ ]:
#Verifica a exixtência de uma tabela específica no banco:

p='ok'
try:
    cur.execute("SELECT name,age FROM person")
    p='ok'
except:
    cur.execute("ROLLBACK")
    p='not ok'
print(p)

In [ ]:
records = cur.fetchall()

In [ ]:
records[0]

In [ ]:
cur.execute("SELECT * FROM person")

In [ ]:
print(records)

In [ ]:
#Buscar registros

p='ok'
try:
    cur.execute("SELECT * FROM getdata_pair")
    records = cur.fetchall()
    p='ok'
except:
    cur.execute("ROLLBACK")
    p='not ok'
print(p)
print('')
print(records)

In [ ]:
#Modelo
http://initd.org/psycopg/docs/usage.html

import psycopg2

try:
    connection = psycopg2.connect(user="",password="", host="", port="", database="")
    print("Selecting rows from mobile table using cursor.fetchall")
    cursor = connection.cursor()
    postgreSQL_select_Query = "select * from mobile"
    cursor.execute(postgreSQL_select_Query)
    mobile_records = cursor.fetchmany(2)

    print("Printing 2 rows")
    
    for row in mobile_records:
        print("Id = ", row[0], )
        print("Model = ", row[1])
       print("Price  = ", row[2], "\n")
   mobile_records = cursor.fetchmany(2)

   print("Printing next 2 rows")
   for row in mobile_records:
       print("Id = ", row[0], )
       print("Model = ", row[1])
       print("Price  = ", row[2], "\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
        


>>> SQL = "INSERT INTO authors (name) VALUES (%s);" # Note: no quotes
cur.execute("INSERT INTO person_teste_marcus (name, age) VALUES (%s, %s)",("marcus", 35))
>>> data = ("O'Reilly", )
>>> cur.execute(SQL, data) # Note: no % operator

# Cria e atualiza tabelas de candles, de acordo com a entrada do usuário django

In [2]:
import psycopg2
import psycopg2.extras
import time
from datetime import datetime
import sys
import subprocess
import requests
import json

In [76]:
# Criando conexão com o banco e ponteiro de consultas

DB_HOST='localhost'
POSTGRES_DB='pg_dev'
POSTGRES_USER='openbot'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'

connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PASSWORD)

pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

### Passo 1: 
    - Buscar na tabela dos pares, o estado de cada par
    - Se não existir esta tabela, descansar 5 min.
    - Se existir tabela:
        - Completa (para t

In [ ]:
#0 - Entrada

pair = "ETHUSDT"
max_requests = '500'

pairs_table_name = 'getdata_pair'
pointer_action = 'SELECT * FROM ' + pairs_table_name

#1 - Busca o registro dos pares em operação
try: 
   
    pointer.execute(pointer_action)
    query_pairs_list = 'ok'
    pairs = pointer.fetchall()
        
    for pair in pairs:
               
        if (pair['status'] == 'ON'):
            
            candles_table_name = 'klines_1m_' + str(pair['pair_name'])
            pointer_action = 'SELECT * FROM ' + candles_table_name
                        
            #2 - verifica se há a tabela de candles, se não existe, a cria:
            try:
                
                pointer.execute(pointer_action)
                query_candle_table = 'ok'

                #2.1 - Adquire os dados históricos, caso não existam:
                if (pair['historical'] == 'absent'):
                    
                    #Historical update
                    pair = str(pair['pair_name'])
                    
                    #pid = subprocess.Popen([sys.executable, "teste.py", pair, max_requests])
                    
                    pid = historical_data(pair, max_requests)
                    
                    print('PID is ', pid)
              
                elif (pair['historical'] == 'full'):
                    
                    candles = pointer.fetchall()
                    #latest_candle = candles[len(candles)]
                    
                    #Current update
                    
            except (Exception, psycopg2.Error) as error:
                
                print(error)
                
                pointer.execute("ROLLBACK")
                query_candle_table = 'not ok' #pode retirar depois
                
                #2.2 - Ou prossegue criando a tabela para este par
                
                pointer_action = "CREATE TABLE " + candles_table_name + ''' 
                (open_time timestamp PRIMARY KEY, open numeric, high numeric, low numeric, close numeric,
                volume numeric, quote_asset_volume numeric, number_of_trades integer, taker_buy_base_asset_volume
                numeric, taker_buy_quote_asset_volume numeric);
                '''
                pointer.execute(pointer_action)
            
except (Exception, psycopg2.Error) as error:
                
    print(error)
    
    print('deu merda')
    pointer.execute("ROLLBACK")
    
    query_pairs_list='not ok'

    time.sleep(10)

In [ ]:
def historical_data(pair, max_requests):
    
    print('par ', pair, ' = ok3')
    
    
    
    
    
    
    
    pid = 2345
    return pid

In [53]:
def get_historical_data(pair, max_requests):
    
    #Atualiza, na tabela "getdata_pairs", os 'historical' dos pares "absent", para "building"
    db_update_status = db_update_pair_table(pair,'building')
    
    klines = []
    num_requests = 0
    
    for i in range(10):
        
        try:

            binance_pair_klines = requests.get('https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime=1241893500000')
            binance_communication = 'ok'
            
            num_requests+= 1
            
            break
            
        except Exception as error:
            
            binance_communication = 'fail'
            print('Comunicação com a binance falhou na entrada, erro: ',error) #Na verdade enviar para destino apropriado

            time.sleep(5)
        
    if (binance_communication == 'ok'):
        
        for i in range(10):
            
            try:
                print('Entrada, gravando no banco')
                klines = klines + binance_pair_klines.json()
                #db_op_status = save_klines_on_db(pair,binance_pair_klines)
                
                break
        
            except Exception as error:
                
                print('Comunicação com o banco de dados falhou na entrada, erro: ',error) #Na verdade enviar para destino apropriado
                
                time.sleep(30)

    #max_requests = 500

    step = 60000
    
    delta_time = 500*step

    start_time = int((binance_pair_klines.json()[0][0])) + delta_time

    ciclo = 0

    while True:

        for i in range(10):

            try:

                request = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime='+str(start_time)

                binance_pair_klines = requests.get(request)

                binance_communication = 'ok'
                
                num_requests+= 1
                
                break

            except Exception as error:

                binance_communication = 'fail'

                print('Comunicação com a binance falhou no cliclo ',ciclo,', tentativa ',i,'. Erro: ', error) #Na verdade enviar para destino apropriado

                time.sleep(5)

        if (binance_communication == 'ok'):
           
            if (len(binance_pair_klines.json()) < 500): #Chegou no mais recente
                
                print('dado mais atual, este foi o último ciclo')
                #Exclui o último dado da binance_pair_klines
            
            for i in range(10):
                
                try:
                    
                    print('Ciclo: ',ciclo,' | tentativa: ',i,': Gravando no banco')
                    klines = klines + binance_pair_klines.json()
                    #db_op_status = save_klines_on_db(pair,binance_pair_klines)
                
                    break
        
                except Exception as error:
                
                    print('Comunicação com o banco de dados falhou no cliclo ',ciclo,', tentativa ',i,'. Erro: ', error) #Na verdade enviar para destino apropriado
                
                    time.sleep(30)
                    
            start_time = start_time + delta_time
                            
            if (len(binance_pair_klines.json()) < 500): 
                
                break

            if (len(binance_pair_klines.json()) > 500): #Alguma anomalia.

                print('ATENÇÃO, MAIS DE 500 AMOSTRAS')
                break
        
        print ('Número de requests: ', num_requests)
        
        if(num_requests > int(max_requests)):

            print('número máximo de requests atingido, dormindo')
            num_requests = 0
            time.sleep(60)
    
        ciclo+=1
    
    #Atualiza, na tabela "getdata_pairs", os 'historical' dos pares "absent", de "building" para "full"
    db_update_status = db_update_pair_table(pair,'full')
    
    return klines

In [77]:
def db_update_pair_table(pair, historical):
    
    #Trocar para os.environment depois
    DB_HOST='localhost'
    POSTGRES_DB='pg_dev'
    POSTGRES_USER='openbot'
    POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'
    
    try:
        
        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)
        
        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
        
        pointer.execute(sql_update_query, (historical, pair))
        connection.commit()
        
        return 'done'
        
    except (Exception, psycopg2.Error) as error:
        
        print("Error in update operation", error)
        
        pointer.execute("ROLLBACK")
        
        return 'fail'
        
    finally:

        if (connection):
            pointer.close()
            connection.close()

In [130]:
def db_save_kline(pair,klines):
    
    candles_table_name = 'klines_1m_' + pair
    
    SQL = '''
    INSERT INTO ''' + candles_table_name + '''
    (open_time, open, high, low, close, volume, 
    quote_asset_volume, number_of_trades, taker_buy_base_asset_volume, 
    taker_buy_quote_asset_volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    try:
        
        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)
        
        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
        
        for kline in klines:
        
            data = (datetime.fromtimestamp(kline[0]/1000), kline[1], kline[2], 
                    kline[3], kline[4], kline[5], kline[7], kline[8], kline[9], kline[10],)
        
            pointer.execute(SQL, data)
        
        return 'done'
        
    except (Exception, psycopg2.Error) as error:
        
        print("Error in update operation", error)
        
        pointer.execute("ROLLBACK")
        
        return 'fail'
        
    finally:

        if (connection):
            
            connection.commit()
            pointer.close()
            connection.close()   

In [131]:
estado = db_save_kline('ADAUSDT',klines)

In [132]:
print (estado)

done


In [ ]:
'''
    
0    1499040000000,      // Open time (miliseconds)
1    "0.01634790",       // Open
2    "0.80000000",       // High
3    "0.01575800",       // Low
4    "0.01577100",       // Close
5    "148976.11427815",  // Volume
6    1499644799999,      // Close time
7    "2434.19055334",    // Quote asset volume
8    308,                // Number of trades
9    "1756.87402397",    // Taker buy base asset volume
10    "28.46694368",      // Taker buy quote asset volume
11   "17928899.62484339" // Ignore.
    
'''

In [123]:
connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                              user=POSTGRES_USER, password=POSTGRES_PASSWORD)

pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

#candles_table_name = 'klines_1m_' + str(pair['pair_name'])
candles_table_name = 'klines_1m_' + 'banbtc'

SQL = '''
INSERT INTO ''' + candles_table_name + '''
(open_time, open, high, low, close, volume, 
quote_asset_volume, number_of_trades, taker_buy_base_asset_volume, 
taker_buy_quote_asset_volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
''' 

# Note: no quotes

for kline in klines:
    
    data = (datetime.fromtimestamp(kline[0]/1000), kline[1], kline[2], kline[3], kline[4], 
            kline[5], kline[7], kline[8], kline[9], kline[10],)
    
    pointer.execute(SQL, data)


connection.commit()

In [101]:
pointer.execute("ROLLBACK")

In [ ]:
import psycopg2
def updateTable(mobileId, price):
    try:
        connection = psycopg2.connect(user="sysadmin",
                                      password="pynative@#29",
                                      host="127.0.0.1",
                                      port="5432",
                                      database="postgres_db")
        cursor = connection.cursor()
        
        print("Table Before updating record ")
        
        sql_select_query = """select * from mobile where id = %s"""
        cursor.execute(sql_select_query, (mobileId, ))
        record = cursor.fetchone()
        print(record)
        
        # Update single record now
        sql_update_query = """Update mobile set price = %s where id = %s"""
        cursor.execute(sql_update_query, (price, mobileId))
        connection.commit()
        
        count = cursor.rowcount
        
        print(count, "Record Updated successfully ")
        print("Table After updating record ")
        
        sql_select_query = """select * from mobile where id = %s"""
        cursor.execute(sql_select_query, (mobileId,))
        record = cursor.fetchone()
        print(record)
    
    except (Exception, psycopg2.Error) as error:
        print("Error in update operation", error)
    
    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
id = 3
price = 970
updateTable(id, price)

In [109]:
db_update_pair_table('BTCUSDT','rola')

'done'

In [107]:

DB_HOST='localhost'
POSTGRES_DB='pg_dev'
POSTGRES_USER='openbot'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'

connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PASSWORD)

pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

sql_select_query = """select * from getdata_pair where pair_name = %s"""
pointer.execute(sql_select_query, ('BTCUSDT', ))
record_old = pointer.fetchone()
print(record_old)

sql_update_query = """Update getdata_pair set historical = %s where pair_name = %s"""
pointer.execute(sql_update_query, ('building', 'BTCUSDT'))
connection.commit()

sql_select_query = """select * from getdata_pair where pair_name = %s"""
pointer.execute(sql_select_query, ('BTCUSDT', ))
record_new = pointer.fetchone()
print(record_new)

['BTCUSDT', 'ON', 'building']
['BTCUSDT', 'ON', 'building']


In [54]:
func_klines = historical_data('BNBBTC', 500)

Entrada, gravando no banco
Ciclo:  0  | tentativa:  0 : Gravando no banco
Número de requests:  2
Ciclo:  1  | tentativa:  0 : Gravando no banco
Número de requests:  3
Ciclo:  2  | tentativa:  0 : Gravando no banco
Número de requests:  4
Ciclo:  3  | tentativa:  0 : Gravando no banco
Número de requests:  5
Ciclo:  4  | tentativa:  0 : Gravando no banco
Número de requests:  6
Ciclo:  5  | tentativa:  0 : Gravando no banco
Número de requests:  7
Ciclo:  6  | tentativa:  0 : Gravando no banco
Número de requests:  8
Ciclo:  7  | tentativa:  0 : Gravando no banco
Número de requests:  9
Ciclo:  8  | tentativa:  0 : Gravando no banco
Número de requests:  10
Ciclo:  9  | tentativa:  0 : Gravando no banco
Número de requests:  11
Ciclo:  10  | tentativa:  0 : Gravando no banco
Número de requests:  12
Ciclo:  11  | tentativa:  0 : Gravando no banco
Número de requests:  13
Ciclo:  12  | tentativa:  0 : Gravando no banco
Número de requests:  14
Ciclo:  13  | tentativa:  0 : Gravando no banco
Número 

Ciclo:  114  | tentativa:  0 : Gravando no banco
Número de requests:  116
Ciclo:  115  | tentativa:  0 : Gravando no banco
Número de requests:  117
Ciclo:  116  | tentativa:  0 : Gravando no banco
Número de requests:  118
Ciclo:  117  | tentativa:  0 : Gravando no banco
Número de requests:  119
Ciclo:  118  | tentativa:  0 : Gravando no banco
Número de requests:  120
Ciclo:  119  | tentativa:  0 : Gravando no banco
Número de requests:  121
Ciclo:  120  | tentativa:  0 : Gravando no banco
Número de requests:  122
Ciclo:  121  | tentativa:  0 : Gravando no banco
Número de requests:  123
Ciclo:  122  | tentativa:  0 : Gravando no banco
Número de requests:  124
Ciclo:  123  | tentativa:  0 : Gravando no banco
Número de requests:  125
Ciclo:  124  | tentativa:  0 : Gravando no banco
Número de requests:  126
Ciclo:  125  | tentativa:  0 : Gravando no banco
Número de requests:  127
Ciclo:  126  | tentativa:  0 : Gravando no banco
Número de requests:  128
Ciclo:  127  | tentativa:  0 : Gravand

Ciclo:  225  | tentativa:  0 : Gravando no banco
Número de requests:  227
Ciclo:  226  | tentativa:  0 : Gravando no banco
Número de requests:  228
Ciclo:  227  | tentativa:  0 : Gravando no banco
Número de requests:  229
Ciclo:  228  | tentativa:  0 : Gravando no banco
Número de requests:  230
Ciclo:  229  | tentativa:  0 : Gravando no banco
Número de requests:  231
Ciclo:  230  | tentativa:  0 : Gravando no banco
Número de requests:  232
Ciclo:  231  | tentativa:  0 : Gravando no banco
Número de requests:  233
Ciclo:  232  | tentativa:  0 : Gravando no banco
Número de requests:  234
Ciclo:  233  | tentativa:  0 : Gravando no banco
Número de requests:  235
Ciclo:  234  | tentativa:  0 : Gravando no banco
Número de requests:  236
Ciclo:  235  | tentativa:  0 : Gravando no banco
Número de requests:  237
Ciclo:  236  | tentativa:  0 : Gravando no banco
Número de requests:  238
Ciclo:  237  | tentativa:  0 : Gravando no banco
Número de requests:  239
Ciclo:  238  | tentativa:  0 : Gravand

Ciclo:  336  | tentativa:  0 : Gravando no banco
Número de requests:  338
Ciclo:  337  | tentativa:  0 : Gravando no banco
Número de requests:  339
Ciclo:  338  | tentativa:  0 : Gravando no banco
Número de requests:  340
Ciclo:  339  | tentativa:  0 : Gravando no banco
Número de requests:  341
Ciclo:  340  | tentativa:  0 : Gravando no banco
Número de requests:  342
Ciclo:  341  | tentativa:  0 : Gravando no banco
Número de requests:  343
Ciclo:  342  | tentativa:  0 : Gravando no banco
Número de requests:  344
Ciclo:  343  | tentativa:  0 : Gravando no banco
Número de requests:  345
Ciclo:  344  | tentativa:  0 : Gravando no banco
Número de requests:  346
Ciclo:  345  | tentativa:  0 : Gravando no banco
Número de requests:  347
Ciclo:  346  | tentativa:  0 : Gravando no banco
Número de requests:  348
Ciclo:  347  | tentativa:  0 : Gravando no banco
Número de requests:  349
Ciclo:  348  | tentativa:  0 : Gravando no banco
Número de requests:  350
Ciclo:  349  | tentativa:  0 : Gravand

Ciclo:  447  | tentativa:  0 : Gravando no banco
Número de requests:  449
Ciclo:  448  | tentativa:  0 : Gravando no banco
Número de requests:  450
Ciclo:  449  | tentativa:  0 : Gravando no banco
Número de requests:  451
Ciclo:  450  | tentativa:  0 : Gravando no banco
Número de requests:  452
Ciclo:  451  | tentativa:  0 : Gravando no banco
Número de requests:  453
Ciclo:  452  | tentativa:  0 : Gravando no banco
Número de requests:  454
Ciclo:  453  | tentativa:  0 : Gravando no banco
Número de requests:  455
Ciclo:  454  | tentativa:  0 : Gravando no banco
Número de requests:  456
Ciclo:  455  | tentativa:  0 : Gravando no banco
Número de requests:  457
Ciclo:  456  | tentativa:  0 : Gravando no banco
Número de requests:  458
Ciclo:  457  | tentativa:  0 : Gravando no banco
Número de requests:  459
Ciclo:  458  | tentativa:  0 : Gravando no banco
Número de requests:  460
Ciclo:  459  | tentativa:  0 : Gravando no banco
Número de requests:  461
Ciclo:  460  | tentativa:  0 : Gravand

...

Ciclo:  1445  | tentativa:  0 : Gravando no banco
Número de requests:  445
Ciclo:  1446  | tentativa:  0 : Gravando no banco
Número de requests:  446
Ciclo:  1447  | tentativa:  0 : Gravando no banco
Número de requests:  447
Ciclo:  1448  | tentativa:  0 : Gravando no banco
Número de requests:  448
Ciclo:  1449  | tentativa:  0 : Gravando no banco
Número de requests:  449
Ciclo:  1450  | tentativa:  0 : Gravando no banco
Número de requests:  450
Ciclo:  1451  | tentativa:  0 : Gravando no banco
Número de requests:  451
Ciclo:  1452  | tentativa:  0 : Gravando no banco
Número de requests:  452
Ciclo:  1453  | tentativa:  0 : Gravando no banco
Número de requests:  453
Ciclo:  1454  | tentativa:  0 : Gravando no banco
Número de requests:  454
Ciclo:  1455  | tentativa:  0 : Gravando no banco
Número de requests:  455
Ciclo:  1456  | tentativa:  0 : Gravando no banco
Número de requests:  456
Ciclo:  1457  | tentativa:  0 : Gravando no banco
Número de requests:  457
Ciclo:  1458  | tentativa

Ciclo:  1555  | tentativa:  0 : Gravando no banco
Número de requests:  54
Ciclo:  1556  | tentativa:  0 : Gravando no banco
Número de requests:  55
Ciclo:  1557  | tentativa:  0 : Gravando no banco
Número de requests:  56
Ciclo:  1558  | tentativa:  0 : Gravando no banco
Número de requests:  57
Ciclo:  1559  | tentativa:  0 : Gravando no banco
Número de requests:  58
Ciclo:  1560  | tentativa:  0 : Gravando no banco
Número de requests:  59
Ciclo:  1561  | tentativa:  0 : Gravando no banco
Número de requests:  60
Ciclo:  1562  | tentativa:  0 : Gravando no banco
Número de requests:  61
Ciclo:  1563  | tentativa:  0 : Gravando no banco
Número de requests:  62
Ciclo:  1564  | tentativa:  0 : Gravando no banco
Número de requests:  63
Ciclo:  1565  | tentativa:  0 : Gravando no banco
Número de requests:  64
Ciclo:  1566  | tentativa:  0 : Gravando no banco
Número de requests:  65
Ciclo:  1567  | tentativa:  0 : Gravando no banco
Número de requests:  66
Ciclo:  1568  | tentativa:  0 : Gravan

Ciclo:  1665  | tentativa:  0 : Gravando no banco
Número de requests:  164
Ciclo:  1666  | tentativa:  0 : Gravando no banco
Número de requests:  165
Ciclo:  1667  | tentativa:  0 : Gravando no banco
Número de requests:  166
Ciclo:  1668  | tentativa:  0 : Gravando no banco
Número de requests:  167
Ciclo:  1669  | tentativa:  0 : Gravando no banco
Número de requests:  168
Ciclo:  1670  | tentativa:  0 : Gravando no banco
Número de requests:  169
Ciclo:  1671  | tentativa:  0 : Gravando no banco
Número de requests:  170
Ciclo:  1672  | tentativa:  0 : Gravando no banco
Número de requests:  171
Ciclo:  1673  | tentativa:  0 : Gravando no banco
Número de requests:  172
Ciclo:  1674  | tentativa:  0 : Gravando no banco
Número de requests:  173
Ciclo:  1675  | tentativa:  0 : Gravando no banco
Número de requests:  174
Ciclo:  1676  | tentativa:  0 : Gravando no banco
Número de requests:  175
Ciclo:  1677  | tentativa:  0 : Gravando no banco
Número de requests:  176
Ciclo:  1678  | tentativa

Ciclo:  1775  | tentativa:  0 : Gravando no banco
Número de requests:  274
Ciclo:  1776  | tentativa:  0 : Gravando no banco
Número de requests:  275
Ciclo:  1777  | tentativa:  0 : Gravando no banco
Número de requests:  276
Ciclo:  1778  | tentativa:  0 : Gravando no banco
Número de requests:  277
Ciclo:  1779  | tentativa:  0 : Gravando no banco
Número de requests:  278
Ciclo:  1780  | tentativa:  0 : Gravando no banco
Número de requests:  279
Ciclo:  1781  | tentativa:  0 : Gravando no banco
Número de requests:  280
Ciclo:  1782  | tentativa:  0 : Gravando no banco
Número de requests:  281
Ciclo:  1783  | tentativa:  0 : Gravando no banco
Número de requests:  282
Ciclo:  1784  | tentativa:  0 : Gravando no banco
Número de requests:  283
Ciclo:  1785  | tentativa:  0 : Gravando no banco
Número de requests:  284
Ciclo:  1786  | tentativa:  0 : Gravando no banco
Número de requests:  285
Ciclo:  1787  | tentativa:  0 : Gravando no banco
Número de requests:  286
Ciclo:  1788  | tentativa

Ciclo:  1885  | tentativa:  0 : Gravando no banco
Número de requests:  384
Ciclo:  1886  | tentativa:  0 : Gravando no banco
Número de requests:  385
Ciclo:  1887  | tentativa:  0 : Gravando no banco
Número de requests:  386
Ciclo:  1888  | tentativa:  0 : Gravando no banco
Número de requests:  387
Ciclo:  1889  | tentativa:  0 : Gravando no banco
Número de requests:  388
Ciclo:  1890  | tentativa:  0 : Gravando no banco
Número de requests:  389
Ciclo:  1891  | tentativa:  0 : Gravando no banco
Número de requests:  390
Ciclo:  1892  | tentativa:  0 : Gravando no banco
Número de requests:  391
Ciclo:  1893  | tentativa:  0 : Gravando no banco
Número de requests:  392
Ciclo:  1894  | tentativa:  0 : Gravando no banco
Número de requests:  393
Ciclo:  1895  | tentativa:  0 : Gravando no banco
Número de requests:  394
Ciclo:  1896  | tentativa:  0 : Gravando no banco
Número de requests:  395
Ciclo:  1897  | tentativa:  0 : Gravando no banco
Número de requests:  396
Ciclo:  1898  | tentativa

In [115]:
print(len(func_klines))

984760


In [117]:
len(func_klines[:(len(func_klines)-1)])

984759

In [75]:
func_klines[600]

[1500040800000,
 '0.00004400',
 '0.00004400',
 '0.00004400',
 '0.00004400',
 '0.00000000',
 1500040859999,
 '0.00000000',
 0,
 '0.00000000',
 '0.00000000',
 '87280650.38596693']

In [63]:
len(func_klines)

984760

In [66]:
datetime.fromtimestamp(1559090340)

datetime.datetime(2019, 5, 28, 21, 39)

In [37]:
#Zona de testes dado histórico
#pair='BNBBTC'
#1 - Localizar dado mais antigo de um par

binance_pair_klines = requests.get('https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime=1241893500000')

oldest_record = int((binance_pair_klines.json()[0][0]))

#print(oldest_record)

#2 - Faz a solicitação do dado de 1min

max_requests = 500

step = 60000

start_time = oldest_record
#start_time =1558728360000
delta_time = 500*step

num_requests = 0

klines = []

ciclo = 0

while True:
    
    for i in range(10):
        
        try:
            
            request = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime='+str(start_time)
            
            binance_pair_klines = requests.get(request)
            
            
            #klines.append(binance_pair_klines.json())
            
            binance_communication = 'ok'
            
            print('tentativa ',i,' do ciclo ',ciclo)
            print('start_time = ',start_time,' ', datetime.fromtimestamp(start_time/1000))
            print('')
            ciclo+=1
            
            #print ("Número de amostras = ", len(binance_pair_klines.json()))
            
            break
            
        except Exception as error:
            
            binance_communication = 'fail'
        
            print(error) #Na verdade enviar para destino apropriado
            
            time.sleep(5)
        
        num_requests+= 1
        
    if(num_requests > int(max_requests)):
        
        time.sleep(60)
            
    if (binance_communication == 'ok'):
        
        klines = klines + binance_pair_klines.json()
        
        start_time = start_time + delta_time
        
        print('tudo certo até aqui')
        
        if (len(binance_pair_klines.json()) > 500): #Alguma anomalia.
            
            #Tratar a exceção
            break
            
        if (len(binance_pair_klines.json()) < 500): #Chegou no mais recente
            
            print('Número de requests = ', num_requests)
            
            break

1500004800000
tentativa  0  do ciclo  0
start_time =  1500004800000   2017-07-14 01:00:00

tudo certo até aqui
tentativa  0  do ciclo  1
start_time =  1500034800000   2017-07-14 09:20:00

tudo certo até aqui
tentativa  0  do ciclo  2
start_time =  1500064800000   2017-07-14 17:40:00

tudo certo até aqui
tentativa  0  do ciclo  3
start_time =  1500094800000   2017-07-15 02:00:00

tudo certo até aqui
tentativa  0  do ciclo  4
start_time =  1500124800000   2017-07-15 10:20:00

tudo certo até aqui
tentativa  0  do ciclo  5
start_time =  1500154800000   2017-07-15 18:40:00

tudo certo até aqui
tentativa  0  do ciclo  6
start_time =  1500184800000   2017-07-16 03:00:00

tudo certo até aqui


KeyboardInterrupt: 

In [45]:
print(datetime.fromtimestamp(int(1500064740)))

2017-07-14 17:39:00


In [ ]:
1498867200000
1502942400000

In [ ]:
step = 60000

start_time = oldest_record

delta_time = 500*(int(max_requests))*step

end_time = start_time + delta_time

binance_pair_klines = requests.get('https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime=1522540800000&endTime=1537540800000')


In [4]:
pair='BNBBTC'

In [5]:
startTime=1558710360000
endTime=1558740300000

#start_time = 1501545600000
#end_time = 

request = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime='+str(startTime)+'&endTime='+str(endTime)
print(request)

binance_pair_klines = requests.get(request)

print(len(binance_pair_klines.json()))

print(datetime.fromtimestamp(int((binance_pair_klines.json()[0][0])/1000)))
print(datetime.fromtimestamp(int((binance_pair_klines.json()[499][0])/1000)))

https://api.binance.com/api/v1/klines?symbol=BNBBTC&interval=1m&startTime=1558710360000&endTime=1558740300000
500
2019-05-24 12:06:00
2019-05-24 20:25:00


In [6]:
print(datetime.fromtimestamp(int(1558710360)))

2019-05-24 12:06:00


In [7]:
print(datetime.fromtimestamp(int(1558740300)))

2019-05-24 20:25:00


In [20]:
inicio = 1558710360000 + 300*60000
         #1559069653
delta_time = 300*60000
fim = inicio + delta_time

print(inicio)
print(' ')
print(fim)
print(' ')

print(datetime.fromtimestamp(int(fim/1000)))


1558728360000
 
1558746360000
 
2019-05-24 22:06:00


In [9]:
datetime.now()

datetime.datetime(2019, 5, 28, 15, 53, 14, 944826)

In [13]:
datetime.timestamp(datetime.utcnow())

1559080530.666529

In [15]:
int(1000*datetime.timestamp(datetime.utcnow()))

1559080563917

In [ ]:
1498867200000

In [12]:
datetime.utcnow()

datetime.datetime(2019, 5, 28, 18, 54, 58, 367711)

In [ ]:
startTime=startTime + (500*60000)
endTime=endTime + (500*60000)

#start_time = 1501545600000
#end_time = 

request = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime='+str(startTime)+'&endTime='+str(endTime)
print(request)

binance_pair_klines = requests.get(request)

print(len(binance_pair_klines.json()))

print(datetime.fromtimestamp(int((binance_pair_klines.json()[0][0])/1000)))
print(datetime.fromtimestamp(int((binance_pair_klines.json()[499][0])/1000)))

In [ ]:
binance_pair_klines.json()

In [24]:
start_time = start_time + (500*60000)

request = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime='+str(start_time)

print(request)

binance_pair_klines = requests.get(request)

print(datetime.fromtimestamp(int((binance_pair_klines.json()[0][0])/1000)))
print(datetime.fromtimestamp(int((binance_pair_klines.json()[499][0])/1000)))

https://api.binance.com/api/v1/klines?symbol=BNBBTC&interval=1m&startTime=1499077200000
2017-07-14 01:00:00
2017-07-14 09:19:00


In [23]:
pair

'BNBBTC'

In [ ]:
(1537540800000 - 1522540800000)/(60*1000)


In [ ]:
len(binance_pair_klines.json())

In [ ]:
len(binance_pair_klines.json())

In [ ]:
type(binance_pair_klines.json())

In [ ]:
type(klines.json())

In [34]:
len(klines)

1500

In [35]:
print(datetime.fromtimestamp(int(1498867200)))

2017-06-30 21:00:00


In [36]:
print(datetime.fromtimestamp(int(1500004800)))

2017-07-14 01:00:00


In [69]:

print(datetime.fromtimestamp(int((func_klines[0][0])/1000)))

2017-07-14 01:00:00


In [70]:
print(datetime.fromtimestamp(int((func_klines[499][0])/1000)))

2017-07-14 09:19:00


In [71]:
print(datetime.fromtimestamp(int((func_klines[500][0])/1000)))

2017-07-14 09:20:00


In [72]:
print(datetime.fromtimestamp(int((func_klines[501][0])/1000)))

2017-07-14 09:21:00


In [73]:
print(datetime.fromtimestamp(int((func_klines[1000][0])/1000)))

2017-07-14 17:40:00


In [ ]:
klines

In [ ]:
step = 60000

start_time = oldest_record

delta_time = 500*(int(max_requests))*step

end_time = start_time + delta_time

new_start_time = end_time + step

print(end_time)

In [ ]:
datetime.fromtimestamp(start_time/1000) #1o candle

In [ ]:
datetime.fromtimestamp(end_time/1000) #1o candle + 1min

In [ ]:
datetime.fromtimestamp(new_start_time/1000) #novo_1o_candle

In [ ]:
i=0
i+= 1
print(i)

In [ ]:
pair

In [ ]:
klines.

In [ ]:
# Agora é preciso repetir o request definindo instantes limites da aquisição

symbol = 'BNBBTC'
granulacao_malha = '1m'
start_time = '1241893500000'
end_time = '1557253500000'

# 1 - Com ambos os limites definidos 
r_1 = requests.get('https://api.binance.com/api/v1/klines?symbol='+symbol+'&interval='+granulacao_malha+'&startTime='+start_time+'&endTime='+end_time)

# 2 - Apenas com limite inferior definido 
r_2 = requests.get('https://api.binance.com/api/v1/klines?symbol='+symbol+'&interval='+granulacao_malha+'&startTime='+start_time)

# 3 - Apenas com limite superior definido 
r_3 = requests.get('https://api.binance.com/api/v1/klines?symbol='+symbol+'&interval='+granulacao_malha+'&endTime='+end_time)

print('Candle mais antigo retornado:')
      
print('> Wrap....:',candles[0])

print ('')

print('> Request.:',r_1.json()[0])

In [ ]:
https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1m&startTime=1501545600&endTime=1501575600

In [ ]:
    1499040000000,      // Open time (miliseconds)
    "0.01634790",       // Open
    "0.80000000",       // High
    "0.01575800",       // Low
    "0.01577100",       // Close
    "148976.11427815",  // Volume
    1499644799999,      // Close time
    "2434.19055334",    // Quote asset volume
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "17928899.62484339" // Ignore.

In [ ]:
exec('python teste.py')

In [ ]:
import subprocess
subprocess.run(["python", "teste.py"])

2+2

In [ ]:
max_requests

In [ ]:
import os
os.system('ts python teste.py')

In [ ]:
import subprocess
import sys

# some code here

pair = "ETHUSDT"
max_requests = '500'

pid = subprocess.Popen([sys.executable, "teste.py", pair, max_requests]) # call subprocess

# some more code here

In [ ]:
print(pid)

In [ ]:
pid.kill()

In [ ]:
print(pid.stdout)

In [ ]:
pid.pid

In [ ]:
subprocess.Popen([sys.executable, "teste.py"]).pid

In [ ]:
pairs[2][1]

print(str(pair[0]) + ' = Completar par')

In [ ]:
len(pairs)

In [ ]:
for pair in pairs:
    print(pair[1])

In [ ]:
pointer.execute("""SELECT * FROM getdata_pair""")
pairs = pointer.fetchall()
print(pairs)

In [ ]:
print(pairs)

In [ ]:
pairs[1]['historical']

In [ ]:
len(pairs)

In [ ]:
pointer.execute("ROLLBACK")

In [ ]:
candles_table_name = 'ETHBTC_1m_klines'
pointer_action = "CREATE TABLE " + candles_table_name +" (name text PRIMARY KEY, age integer);"
pointer.execute(pointer_action)

In [ ]:
candles_table_name = 'klines_1m_BANBTC'

pointer_action = "CREATE TABLE " + candles_table_name + ''' 
(open_time timestamp PRIMARY KEY, open numeric, high numeric, low numeric, close numeric,
volume numeric, quote_asset_volume numeric, number_of_trades integer, taker_buy_base_asset_volume numeric,
taker_buy_quote_asset_volume numeric);
'''

pointer.execute(pointer_action)

In [ ]:
candles_table_name = 'klines_1m_BANBTC'

SQL = "INSERT INTO " + candles_table_name + ''' (open_time, open, high, low, close, volume, 
quote_asset_volume, number_of_trades, taker_buy_base_asset_volume, taker_buy_quote_asset_volume) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
'''

#cur.execute("INSERT INTO person_teste_marcus (name, age) VALUES (%s, %s)",("marcus", 35))
data = (datetime.fromtimestamp(1499040000), 0.01634790, 0.80000000, 0.01575800, 0.01577100, 148976.11427815, 2434.19055334, 308, 1756.87402397, 28.46694368)
pointer.execute(SQL, data)


In [ ]:
for pair in pairs:
    print(pair['status'])